In [ ]:
import win32com.client
import datetime

tags = ["Applications.Application1.tag1:In,Log1",
        "Applications.Application1.tag2:In,Log1"]

# Allow gencache to create the cached wrapper objects
win32com.client.gencache.is_readonly = False

# Under p2exe the call in gencache to __init__() does not happen
# so we use Rebuild() to force the creation of the gen_py folder
win32com.client.gencache.Rebuild(verbose=0)

#_opchda = win32com.client.gencache.EnsureDispatch('Graybox.OPC.HDAWrapper', 0)
_opchda = win32com.client.gencache.EnsureDispatch('ABBOPCHDA.Automation', 0)

_opchda.Connect('ABB.AdvHtHistorySrv.1', 'localhost')
opcHDAItems = _opchda.OPCHDAItems

NumItems = len(tags)

names = [0]
names.extend(tags)
clientHandles = list(range(NumItems))
clientHandles.insert(0,0)

try:
    errors = opcHDAItems.Validate(NumItems, names)
except:
    pass

try:
    server_handles, errors = opcHDAItems.AddItems(NumItems, names, clientHandles)
except:
    pass

s_handles = list(server_handles)
s_handles.insert(0,0)

StartTime = datetime.datetime.strptime('2021-01-01 01:00:00.00','%Y-%m-%d %H:%M:%S.%f')
EndTime = "NOW"
NumValues = 0
Bounds = False

ItemValues = opcHDAItems.SyncReadRaw(StartTime, EndTime, NumValues, Bounds, NumItems, s_handles)

datum1 = ItemValues[0]
datum2 = ItemValues[1]
obj = ItemValues[2]

#print(obj[0].Count)
#print(obj[1].Item)
n=0
for i, hdaHistoryInstance in enumerate(obj):
    for val in hdaHistoryInstance:
        print(f"{val.TimeStamp.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]} - {val}")
